In [1]:
%pip install -q llama-index chromadb loguru pypdf openai sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import openai

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

In [3]:
!wget -O Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876

--2024-02-07 18:11:37--  https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
Resolving dl.dropbox.com (dl.dropbox.com)... 162.125.81.15, 2620:100:6031:15::a27d:510f
Connecting to dl.dropbox.com (dl.dropbox.com)|162.125.81.15|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876 [following]
--2024-02-07 18:11:38--  https://dl.dropboxusercontent.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.81.15, 2620:100:6031:15::a27d:510f
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.81.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221928 (217K) [app

In [4]:
FILE_NAME = "./Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf"

In [5]:
import os
from pathlib import Path
from typing import List

import chromadb
from chromadb.api.models.Collection import Collection
from llama_index import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    get_response_synthesizer,
)
import pickle

from llama_index.response_synthesizers.base import BaseSynthesizer
from llama_index.response_synthesizers.type import ResponseMode
from llama_index.postprocessor.types import BaseNodePostprocessor
from llama_index.indices.postprocessor import (
    MetadataReplacementPostProcessor,
    SentenceTransformerRerank,
)

from llama_index.agent.types import BaseAgent
from llama_index.core.base_query_engine import BaseQueryEngine
from llama_index.core.base_retriever import BaseRetriever
from llama_index.agent import OpenAIAgent
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.embeddings.utils import EmbedType
from llama_index.indices.base import BaseIndex
from llama_index.indices.document_summary import DocumentSummaryIndex
from llama_index.llms import OpenAI
from llama_index.llms.utils import LLMType
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.response.schema import RESPONSE_TYPE
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.vector_stores import ChromaVectorStore
from loguru import logger
from pydantic import FilePath
from llama_index.indices.document_summary import (
    DocumentSummaryIndex,
    DocumentSummaryIndexLLMRetriever,
)

In [6]:
TEMPERATURE = 0.0
SIM_TOP_K = 3
RERANK_TOP_K = 3
WIN_SZ = 3

In [7]:
def create_vectors(path: str, collection_name="tmp_collection") -> Collection:
    chroma_client = chromadb.PersistentClient(path)
    # https://github.com/run-llama/llama_index/issues/6528
    return chroma_client.get_or_create_collection(collection_name)

In [8]:
def create_service_context(llm: LLMType, embs: EmbedType) -> ServiceContext:
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=WIN_SZ,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )

    return ServiceContext.from_defaults(
        node_parser=node_parser,
        llm=llm,
        embed_model=embs,
    )


def create_storage_context() -> StorageContext:
    path: str = "./db/LlamaIndexMultiVectorSummaryAgent"
    return StorageContext.from_defaults(
        vector_store=ChromaVectorStore(create_vectors(path=path))
    )

def get_summary(summary_query_engine: BaseQueryEngine, saved_summary_path: str="./summary_output.pkl") -> str:
    if not os.path.exists(saved_summary_path):
        Path(saved_summary_path).parent.mkdir(parents=True, exist_ok=True)
        summary = str(
            summary_query_engine.query(
                "Extract a concise 1-2 line summary of this document"
            )
        )
        pickle.dump(summary, open(saved_summary_path, "wb"))
    else:
        summary: str = pickle.load(open(saved_summary_path, "rb"))

    return summary

def from_retriever_to_query_engine(
    service_context: ServiceContext,
    retriever: BaseRetriever,
) -> BaseQueryEngine:
    postproc: BaseNodePostprocessor = MetadataReplacementPostProcessor(
        target_metadata_key="window"
    )
    rerank: BaseNodePostprocessor = SentenceTransformerRerank(
        top_n=RERANK_TOP_K, model="BAAI/bge-reranker-base"
    )
    response_synthesizer: BaseSynthesizer = get_response_synthesizer(
        service_context=service_context,
        response_mode=ResponseMode.REFINE,
    )
    return RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer,
        node_postprocessors=[postproc, rerank],
    )

In [9]:
llm = OpenAI(model="gpt-4-1106-preview", temperature=TEMPERATURE)
embs = "local:BAAI/bge-small-en-v1.5"

service_context: ServiceContext = (
    create_service_context(llm, embs)
)

storage_context: StorageContext = (
    create_storage_context()
)

input_files: List[str] = [FILE_NAME]
docs: SimpleDirectoryReader = SimpleDirectoryReader(
    input_files=input_files,
).load_data()

logger.debug("Start loading document index from storage")
summary_index: BaseIndex = DocumentSummaryIndex.from_documents(
    docs,
    storage_context=storage_context,
    service_context=service_context,
    show_progress=True,
)

vector_index: BaseIndex = VectorStoreIndex.from_documents(
    docs,
    service_context=service_context,
    storage_context=storage_context,
    show_progress=True,
)
logger.debug("Finish loading document index from storage")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

current doc id: d708cc51-9101-4490-8973-079b974a145b
current doc id: 89c08c78-a0b5-480a-8381-3790e081080e
current doc id: 83d16658-f51d-4de1-a194-57ba742d0815
current doc id: 511e302f-2816-4ce9-9353-38d170616766
current doc id: 2b3efb45-13eb-4deb-8666-aabc5bce636c
current doc id: 89a08310-38ee-45a8-a2df-35dfc8696596
current doc id: 8c870af8-461f-4c8b-b634-966721b013bd
current doc id: b3893794-2ae7-4643-b0df-5843d70180a8
current doc id: c54fed37-d3e7-4b99-890e-018f83ad43a0
current doc id: 06e89dd8-4a69-457a-a932-09a6ebb15201


Generating embeddings:   0%|          | 0/10 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/160 [00:00<?, ?it/s]

2024-02-07 18:15:22.622 | DEBUG    | __main__:<cell line: 31>:31 - Finish loading document index from storage


In [10]:
logger.debug("Start creating agent with tools")

summary_query_engine: BaseQueryEngine = from_retriever_to_query_engine(
            service_context=service_context,
            retriever=DocumentSummaryIndexLLMRetriever(
                summary_index,
                similarity_top_k=SIM_TOP_K,
            ),
)

vector_query_engine: BaseQueryEngine = from_retriever_to_query_engine(
    service_context=service_context,
    retriever=vector_index.as_retriever(
        similarity_top_k=SIM_TOP_K
    ),
)

query_engine_tools = [
    QueryEngineTool(
        query_engine=summary_query_engine,
        metadata=ToolMetadata(
            name="summary_tool",
            description=f"Useful for summarization questions on the `{FILE_NAME}` document",
        ),
    ),
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="vector_tool",
            description=f"Useful for questions related to specific facts in the `{FILE_NAME}` document",
        ),
    ),
]
agent: BaseAgent = OpenAIAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    system_prompt=f"""\
You are a specialized agent designed to answer queries about the `{FILE_NAME}` document.
You must ALWAYS use at least ONE of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
)
logger.debug("Finish creating agent with tools")

2024-02-07 18:15:22.630 | DEBUG    | __main__:<cell line: 1>:1 - Start creating agent with tools


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

2024-02-07 18:15:34.697 | DEBUG    | __main__:<cell line: 43>:43 - Finish creating agent with tools


# Query

The settings of

SIM_TOP_K = 3

RERANK_TOP_K = 3

can affect the query duration.

In [11]:
prompt = """What is the topic of content?
    The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
    """
result: RESPONSE_TYPE = agent.query(prompt)

Added user message to memory: What is the topic of content?
    The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
    
=== Calling Function ===
Calling function: summary_tool with args: {"input":"What is the topic of content?"}
Got output: The content discusses Baidu Inc.'s Code of Business Conduct and Ethics, highlighting the responsibility of all employees to be aware of and adhere to the code. It outlines the obligation of employees to comply with applicable laws and regulations in various jurisdictions, including those related to securities, labor, employment, anti-bribery, intellectual property, privacy, insider trading, gratuities, harassment, health and safety, and the prevention of financial misinformation or misuse of corporate assets. The code is applicable across all levels of the company's hierarchy.



In [12]:
from IPython.display import display, HTML

display(HTML(result.response))